In [ ]:
# Parameter Optimization for Trading Profiles
# This notebook tests different parameter combinations and exports results to CSV

import sys
from pathlib import Path
import pandas as pd
import numpy as np
from itertools import product
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import time
import warnings
import os

def find_project_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "src").is_dir():
            return p
    return start

CWD = Path.cwd()
PROJECT_ROOT = find_project_root(CWD)
SRC_PATH = PROJECT_ROOT / "src"
for p in (PROJECT_ROOT, SRC_PATH):
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
os.environ["PYTHONPATH"] = os.pathsep.join({str(PROJECT_ROOT), str(SRC_PATH)})  # help some kernels pick it up

warnings.filterwarnings('ignore')

print(f"Working dir: {CWD}")
print(f"Project root: {PROJECT_ROOT}")
print(f"src path: {SRC_PATH}")
print("✅ Imports successful")

Working dir: c:\Users\rastr\Documents\repositories\trading-bot-project\notebooks
Project root: c:\Users\rastr\Documents\repositories\trading-bot-project
src path: c:\Users\rastr\Documents\repositories\trading-bot-project\src
✅ Imports successful


In [68]:
# Import strategy components
from src.ingestion.data_fetcher import DataFetcher
from src.metrics.technical import TechnicalIndicators
from src.models.ml_model import MLModel
from src.metrics.backtester import Backtester
from src.metrics.performance import PerformanceMetrics

print("✅ Strategy modules imported")

✅ Strategy modules imported


In [69]:
# Load data once (reuse for all parameter combinations)
print("Fetching 730 days of BTC/USDT 1h data...")
df = DataFetcher.fetch_ohlcv("BTC/USDT", "1h", 730)
print(f"✅ Loaded {len(df)} candles")

# Split into train/test (70/30)
train_size = int(len(df) * 0.7)
df_train = df.iloc[:train_size]
df_test = df.iloc[train_size:]

print(f"Train: {len(df_train)} bars, Test: {len(df_test)} bars")

2025-11-02 22:01:15 - INFO - Fetching 730 days of 1h data for BTC/USDT


Fetching 730 days of BTC/USDT 1h data...


2025-11-02 22:07:10 - INFO - Fetched 17521 candles


✅ Loaded 17521 candles
Train: 12264 bars, Test: 5257 bars


In [70]:
# Define parameter ranges to test
# Adjust these ranges based on what you want to optimize

param_grid = {
    # SMA periods
    'sma_fast': [35],
    'sma_slow': [65],
    
    # Indicator parameters (tinker freely)
    'volume_ma_period': [24, 10, 15],
    'rsi_period': [14, 7],
    'bb_period': [7, 15, 10],
    'bb_std': [2.0],
    
    # ML threshold (minimum predicted gain to enter trade)
    'ml_threshold': [0.01],  

    # Exit parameters
    'profit_target': [0.07, 0.075, 0.8, 0.9],
    'stop_loss': [0.02, 0.025],
    
    # Volume filter
    'require_volume': [True],
    
    # Max hold time (hours)
    'max_hold_hours': [120, 96],
}

# Calculate total combinations
total_combinations = np.prod([len(v) for v in param_grid.values()])
print(f"Total parameter combinations to test: {total_combinations:,}")
print(f"\nParameter ranges:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

Total parameter combinations to test: 288

Parameter ranges:
  sma_fast: [35]
  sma_slow: [65]
  volume_ma_period: [24, 10, 15]
  rsi_period: [14, 7]
  bb_period: [7, 15, 10]
  bb_std: [2.0]
  ml_threshold: [0.01]
  profit_target: [0.07, 0.075, 0.8, 0.9]
  stop_loss: [0.02, 0.025]
  require_volume: [True]
  max_hold_hours: [120, 96]


In [71]:
# Optimization function
def test_parameters(params, df_train, df_test):
    """
    Test a single parameter combination
    
    Returns dict with parameters and performance metrics
    Includes indicator parameters from params: volume_ma_period, rsi_period, bb_period, bb_std.
    """
    # Build config from parameters
    # indicator_config = {
    #     'sma_fast': params['sma_fast'],
    #     'sma_slow': params['sma_slow'],
    #     'volume_ma_period': 20,
    #     'rsi_period': 14,
    #     'bb_period': 20,
    #     'bb_std': 2,
    # }

    indicator_config = {
        'sma_fast': params['sma_fast'],
        'sma_slow': params['sma_slow'],
        'volume_ma_period': params['volume_ma_period'],
        'rsi_period': params['rsi_period'],
        'bb_period': params['bb_period'],
        'bb_std': params['bb_std'],
    }
    
    profile_config = {
        'market': {'symbol': 'BTC/USDT', 'timeframe': '1h'},
        'indicators': indicator_config,
        'ml': {
            'enabled': True,
            'prediction_horizon': 48,
            'threshold': params['ml_threshold'],
            'model_params': {
                'n_estimators': 200,
                'max_depth': 4,
                'learning_rate': 0.05,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'random_state': 42,
            }
        },
        'entry': {
            'require_sma_cross': True,
            'require_volume': params['require_volume'],
            'require_ml': True,
            'entry_fee': 0.001,
        },
        'exit': {
            'profit_target': params['profit_target'],
            'stop_loss': params['stop_loss'],
            'max_hold_hours': params['max_hold_hours'],
            'exit_on_sma_cross_down': True,
            'exit_fee': 0.001,
        },
    }
    
    output_config = {'base_path': '../output', 'save_model': False}
    
    try:
        # Calculate indicators
        df_train_ind = TechnicalIndicators.calculate_all(df_train.copy(), indicator_config)
        df_test_ind = TechnicalIndicators.calculate_all(df_test.copy(), indicator_config)
        
        # Train ML model
        ml_model = MLModel(profile_config['ml'], output_config, 'OPTIMIZE')
        ml_model.train(df_train_ind)
        ml_predictions = ml_model.predict(df_test_ind)
        
        # Run backtest
        backtester = Backtester(profile_config, ml_predictions)
        trades_df, signals_df, final_equity = backtester.run(df_test_ind)
        
        # Calculate metrics
        metrics = PerformanceMetrics.calculate(trades_df, profile_config)
        
        if not metrics:
            return None
        
        # Calculate projections with $10K starting capital
        net_return_pct = metrics['net_return'] * 100
        starting_capital = 10000
        ending_balance = starting_capital * (1 + metrics['net_return'])
        net_profit = ending_balance - starting_capital
        
        # Combine parameters and results
        result = {
            **params,  # All parameters
            'net_return_pct': round(net_return_pct, 2),
            'annualized_pct': round((metrics['net_return'] / (len(df_test_ind) / 24)) * 365 * 100, 1),
            'total_trades': metrics['num_trades'],
            'win_rate_pct': round(metrics['win_rate'] * 100, 1),
            'profit_factor': round(metrics['profit_factor'], 2),
            'sharpe_ratio': round(metrics['sharpe_ratio'], 2),
            'max_drawdown_pct': round(metrics['max_drawdown'] * 100, 2),
            'avg_win_pct': round(metrics['avg_win'] * 100, 2),
            'avg_loss_pct': round(metrics['avg_loss'] * 100, 2),
            # $10K projections
            'profit_from_10k': round(net_profit, 2),
            'ending_balance_10k': round(ending_balance, 2),
        }
        
        return result
        
    except Exception as e:
        print(f"Error with params {params}: {str(e)}")
        return None

print("✅ Optimization function defined")

✅ Optimization function defined


In [72]:
# Run optimization (PARALLEL - uses all CPU cores)
# Set n_jobs to specific number if you want to limit cores (e.g., n_jobs=4)
# Set n_jobs=-1 to use all available cores

import multiprocessing
n_cores = multiprocessing.cpu_count()
print(f"🚀 Using {n_cores} CPU cores for parallel processing\n")

param_combinations = list(product(*param_grid.values()))
param_keys = list(param_grid.keys())

print(f"Starting optimization of {len(param_combinations):,} combinations...")

# Time a small sample to estimate total time
print("\n⏱️  Running sample batch to estimate completion time...")
sample_size = min(n_cores * 2, len(param_combinations))
sample_start = time.time()

sample_results = Parallel(n_jobs=-1, verbose=0)(
    delayed(test_parameters)(dict(zip(param_keys, values)), df_train, df_test)
    for values in param_combinations[:sample_size]
)

sample_time = time.time() - sample_start
avg_time_per_combo = sample_time / sample_size
estimated_total_time = (avg_time_per_combo * len(param_combinations)) / n_cores

print(f"✅ Sample completed in {sample_time:.1f}s")
print(f"📊 Average time per combination: {avg_time_per_combo:.2f}s")
print(f"⏰ Estimated total time: {estimated_total_time/60:.1f} minutes")
print(f"   (Actual time may vary based on CPU load)\n")

# Run full optimization with joblib's built-in progress
print(f"Running full optimization...")
print(f"Progress will be shown every ~{max(1, len(param_combinations)//20)} batches\n")
start_time = time.time()

# verbose=10 shows progress every 10 seconds
# batch_size='auto' optimizes for your CPU
results = Parallel(n_jobs=-1, verbose=10, batch_size='auto')(
    delayed(test_parameters)(dict(zip(param_keys, values)), df_train, df_test)
    for values in param_combinations
)

# Filter out None results (failed runs)
results = [r for r in results if r is not None]

elapsed_time = time.time() - start_time
print(f"\n" + "="*80)
print(f"✅ OPTIMIZATION COMPLETE!")
print(f"="*80)
print(f"Total time: {elapsed_time/60:.1f} minutes ({elapsed_time:.0f} seconds)")
print(f"Successfully tested: {len(results):,} / {len(param_combinations):,} combinations")
print(f"Success rate: {len(results)/len(param_combinations)*100:.1f}%")
print(f"Average time per combination: {elapsed_time/len(param_combinations):.2f}s")
print(f"="*80 + "\n")

# Convert to DataFrame
results_df = pd.DataFrame(results)
print(f"Results DataFrame shape: {results_df.shape}")
print(f"Columns: {list(results_df.columns)}\n")

🚀 Using 20 CPU cores for parallel processing

Starting optimization of 288 combinations...

⏱️  Running sample batch to estimate completion time...
✅ Sample completed in 5.1s
📊 Average time per combination: 0.13s
⏰ Estimated total time: 0.0 minutes
   (Actual time may vary based on CPU load)

Running full optimization...
Progress will be shown every ~14 batches

✅ Sample completed in 5.1s
📊 Average time per combination: 0.13s
⏰ Estimated total time: 0.0 minutes
   (Actual time may vary based on CPU load)

Running full optimization...
Progress will be shown every ~14 batches



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:  


✅ OPTIMIZATION COMPLETE!
Total time: 0.2 minutes (13 seconds)
Successfully tested: 288 / 288 combinations
Success rate: 100.0%
Average time per combination: 0.04s

Results DataFrame shape: (288, 22)
Columns: ['sma_fast', 'sma_slow', 'volume_ma_period', 'rsi_period', 'bb_period', 'bb_std', 'ml_threshold', 'profit_target', 'stop_loss', 'require_volume', 'max_hold_hours', 'net_return_pct', 'annualized_pct', 'total_trades', 'win_rate_pct', 'profit_factor', 'sharpe_ratio', 'max_drawdown_pct', 'avg_win_pct', 'avg_loss_pct', 'profit_from_10k', 'ending_balance_10k']



[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:   12.6s finished


In [75]:
# Export filtered top performers to CSV (append)
output_path = PROJECT_ROOT / "output" / "optimization_results.csv"

# Ensure output folder exists
output_path.parent.mkdir(parents=True, exist_ok=True)

# Filter: win_rate_pct > 50 and total_trades >= 7; then take Top 10 by annualized return
filtered_top = (
    results_df[
        (results_df['win_rate_pct'] > 50) &
        (results_df['total_trades'] >= 7)
    ]
    .sort_values('annualized_pct', ascending=False)
    .head(10)
 )

# Append to CSV (write header only if file doesn't exist)
write_header = not output_path.exists()
filtered_top.to_csv(output_path, index=False, mode='a', header=write_header)

print(f"✅ Appended {len(filtered_top)} rows to: {output_path}")
print("Filter applied: win_rate_pct > 50, total_trades >= 7; sorted by annualized_pct desc; top 10")
display(filtered_top)

✅ Appended 10 rows to: c:\Users\rastr\Documents\repositories\trading-bot-project\output\optimization_results.csv
Filter applied: win_rate_pct > 50, total_trades >= 7; sorted by annualized_pct desc; top 10


,sma_fast,sma_slow,volume_ma_period,rsi_period,bb_period,bb_std,ml_threshold,profit_target,stop_loss,require_volume,...,annualized_pct,total_trades,win_rate_pct,profit_factor,sharpe_ratio,max_drawdown_pct,avg_win_pct,avg_loss_pct,profit_from_10k,ending_balance_10k
239,35,65,15,14,10,2.0,0.01,0.9,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
235,35,65,15,14,10,2.0,0.01,0.8,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
251,35,65,15,7,7,2.0,0.01,0.8,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
255,35,65,15,7,7,2.0,0.01,0.9,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
207,35,65,15,14,7,2.0,0.01,0.9,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
203,35,65,15,14,7,2.0,0.01,0.8,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
287,35,65,15,7,10,2.0,0.01,0.9,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
283,35,65,15,7,10,2.0,0.01,0.8,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
219,35,65,15,14,15,2.0,0.01,0.8,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
223,35,65,15,14,15,2.0,0.01,0.9,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01


In [76]:
# Quick analysis - Top performers
print("=" * 80)
print("TOP 10 by Annualized Return")
print("=" * 80)
top_annualized = results_df.nlargest(10, 'annualized_pct')
display(top_annualized)

print("\n" + "=" * 80)
print("TOP 10 by Sharpe Ratio")
print("=" * 80)
top_sharpe = results_df.nlargest(10, 'sharpe_ratio')
display(top_sharpe)

TOP 10 by Annualized Return


,sma_fast,sma_slow,volume_ma_period,rsi_period,bb_period,bb_std,ml_threshold,profit_target,stop_loss,require_volume,...,annualized_pct,total_trades,win_rate_pct,profit_factor,sharpe_ratio,max_drawdown_pct,avg_win_pct,avg_loss_pct,profit_from_10k,ending_balance_10k
203,35,65,15,14,7,2.0,0.01,0.8,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
207,35,65,15,14,7,2.0,0.01,0.9,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
219,35,65,15,14,15,2.0,0.01,0.8,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
223,35,65,15,14,15,2.0,0.01,0.9,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
235,35,65,15,14,10,2.0,0.01,0.8,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
239,35,65,15,14,10,2.0,0.01,0.9,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
251,35,65,15,7,7,2.0,0.01,0.8,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
255,35,65,15,7,7,2.0,0.01,0.9,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
283,35,65,15,7,10,2.0,0.01,0.8,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
287,35,65,15,7,10,2.0,0.01,0.9,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01



TOP 10 by Sharpe Ratio


,sma_fast,sma_slow,volume_ma_period,rsi_period,bb_period,bb_std,ml_threshold,profit_target,stop_loss,require_volume,...,annualized_pct,total_trades,win_rate_pct,profit_factor,sharpe_ratio,max_drawdown_pct,avg_win_pct,avg_loss_pct,profit_from_10k,ending_balance_10k
267,35,65,15,7,15,2.0,0.01,0.800,0.025,True,...,62.0,15,53.3,4.93,9.39,-3.22,5.55,-1.27,3672.73,13672.73
271,35,65,15,7,15,2.0,0.01,0.900,0.025,True,...,62.0,15,53.3,4.93,9.39,-3.22,5.55,-1.27,3672.73,13672.73
263,35,65,15,7,15,2.0,0.01,0.075,0.025,True,...,57.9,15,53.3,4.70,9.32,-3.22,5.30,-1.27,3429.85,13429.85
259,35,65,15,7,15,2.0,0.01,0.070,0.025,True,...,56.4,15,53.3,4.62,9.31,-3.22,5.22,-1.27,3341.99,13341.99
265,35,65,15,7,15,2.0,0.01,0.800,0.020,True,...,61.1,15,53.3,4.76,9.23,-3.22,5.55,-1.32,3621.88,13621.88
269,35,65,15,7,15,2.0,0.01,0.900,0.020,True,...,61.1,15,53.3,4.76,9.23,-3.22,5.55,-1.32,3621.88,13621.88
203,35,65,15,14,7,2.0,0.01,0.800,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
207,35,65,15,14,7,2.0,0.01,0.900,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
219,35,65,15,14,15,2.0,0.01,0.800,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
223,35,65,15,14,15,2.0,0.01,0.900,0.025,True,...,62.8,16,56.2,4.99,9.16,-3.22,4.98,-1.27,3720.01,13720.01
